<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/quantum_price_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane
import pennylane as qml
from pennylane import numpy as np
import yfinance as yf

# We start by defining a simple quantum device with 1 qubit
dev = qml.device("default.qubit", wires=1)

# Define the quantum circuit
@qml.qnode(dev)
def circuit(params, x):
    qml.RX(x, wires=0)
    qml.RY(params[0], wires=0)
    return qml.expval(qml.PauliZ(0))

# Define the cost function (mean squared error)
def cost(params, x, y):
    predictions = np.array([circuit(params, x_) for x_ in x])
    return np.mean((predictions - y) ** 2)

# Download historical stock data
data = yf.download('AAPL', '2020-01-01', '2021-12-31')

# We will use the close prices to train the model
prices = data['Close'].values

# Normalize the prices to the range [0, pi/2] to be compatible with the quantum circuit
prices_normalized = np.pi/2 * (prices - np.min(prices)) / (np.max(prices) - np.min(prices))

# Split data into train and test datasets
train_prices, test_prices = np.split(prices_normalized, [int(.7 *len(prices_normalized))])
train_dates, test_dates = np.split(data.index, [int(.7 *len(data.index))])

# Initialize some random parameters
params = np.random.random(1)

# Perform gradient descent to train the model
for i in range(100):
    params -= 0.1 * qml.grad(cost)(params, train_prices[:-1], train_prices[1:])
    print("Step", i, "cost", cost(params, train_prices[:-1], train_prices[1:]))

print("Optimized parameters", params)

# Use the trained model to make predictions on the test data
predictions = np.array([circuit(params, x) for x in test_prices[:-1]])

# Interpret the predictions as "buy" or "sell" signals
for i in range(len(predictions)):
    if predictions[i] > test_prices[i]:
        print(f"Date {test_dates[i].strftime('%m/%d/%Y')}: BUY")
    else:
        print(f"Date {test_dates[i].strftime('%m/%d/%Y')}: SELL")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 98.5 MB/s eta 0:00:00
[*********************100%***********************]  1 of 1 completed
Step 0 cost 0.17633135042170794
Step 1 cost 0.1763184203317351
Step 2 cost 0.17630724345917007
Step 3 cost 0.17629757779697236
Step 4 cost 0.17628921556820557
Step 5 cost 0.17628197822582142
Step 6 cost 0.17627571221332947
Step 7 cost 0.17627028536473152
Step 8 cost 0.17626558384265487
Step 9 cost 0.17626150953048453
Step 10 cost 0.17625797780818167
